In [2]:
%cd ..
import os, sys
sys.path.append(os.path.join(os.getcwd(), "src"))

d:\Blockchain\data\orai\orchai_validator


In [3]:
from orchai.upload import get_spark, sampling, uploading, run_uploading
from orchai.constants import Constants
from orchai.etl_processor import validating, ETLProcessor
from omegaconf import OmegaConf
import pyspark.sql.functions as F
from pyspark.sql import DataFrame, Window
import numpy as np

In [5]:
spark = get_spark()
config = OmegaConf.load("config/etl_file_1m.yaml")
sb, eb = 7059473, 9583823
print((eb - sb) / 150 + 1)

16830.0


In [6]:
run_uploading(config, sb, eb, spark, cal_score=True, overwrite=True)

| Intervals:
0 - 7059473 -> 7659473 : 4001.0
1 - 7213373 -> 7813373 : 4001.0
2 - 7367273 -> 7967273 : 4001.0
3 - 7521173 -> 8121173 : 4001.0
4 - 7675073 -> 8275073 : 4001.0
5 - 7828973 -> 8428973 : 4001.0
6 - 7982873 -> 8582873 : 4001.0
7 - 8136773 -> 8736773 : 4001.0
8 - 8290673 -> 8890673 : 4001.0
9 - 8444573 -> 9044573 : 4001.0
10 - 8598473 -> 9198473 : 4001.0
11 - 8752373 -> 9352373 : 4001.0
12 - 8906273 -> 9506273 : 4001.0
13 - 9060173 -> 9583823 : 3492.0
| Start
0 - 7059473 -> 7659473 : 4001.0
Successfully query data from database
------------------------------------------------
Successfully converted vote_propose_score column
start-end final blocks: 7073723 7659473
------------------------------------------------
Successfully converted voting_power_score column
start-end final blocks: 7073723 7659473
------------------------------------------------
Successfully converted commission_score column
start-end final blocks: 7073723 7659473
---------------------------------------------